# LangGraph(함수형 API)를 AutoGen, CrewAI 및 기타 프레임워크와 통합하는 방법

LangGraph는 에이전틱 및 멀티 에이전트 애플리케이션을 구축하기 위한 프레임워크입니다. LangGraph는 다른 에이전트 프레임워크와 쉽게 통합할 수 있습니다.

LangGraph를 다른 에이전트 프레임워크와 통합하려는 주요 이유:

- 개별 에이전트가 다른 프레임워크로 구축된 [멀티 에이전트 시스템](../../concepts/multi_agent) 생성
- LangGraph를 활용하여 [지속성](../../concepts/persistence), [스트리밍](../../concepts/streaming), [단기 및 장기 메모리](../../concepts/memory) 등과 같은 기능 추가

다른 프레임워크의 에이전트를 통합하는 가장 간단한 방법은 LangGraph [노드](../../concepts/low_level/#nodes) 내에서 해당 에이전트를 호출하는 것입니다:

```python
import autogen
from langgraph.func import entrypoint, task

autogen_agent = autogen.AssistantAgent(name="assistant", ...)
user_proxy = autogen.UserProxyAgent(name="user_proxy", ...)

@task
def call_autogen_agent(messages):
    response = user_proxy.initiate_chat(
        autogen_agent,
        message=messages[-1],
        ...
    )
    ...


@entrypoint()
def workflow(messages):
    response = call_autogen_agent(messages).result()
    return response


workflow.invoke(
    [
        {
            "role": "user",
            "content": "Find numbers between 10 and 30 in fibonacci sequence",
        }
    ]
)
```

이 가이드에서는 AutoGen과 통합되는 LangGraph 챗봇을 구축하는 방법을 보여주지만 다른 프레임워크에도 동일한 접근 방식을 따를 수 있습니다.

## 설정

In [1]:
%pip install autogen langgraph

In [2]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("OPENAI_API_KEY")

OPENAI_API_KEY:  ········


## AutoGen 에이전트 정의

여기서 AutoGen 에이전트를 정의합니다. [여기](https://github.com/microsoft/autogen/blob/0.2/notebook/agentchat_web_info.ipynb)의 공식 튜토리얼에서 수정했습니다.

In [ ]:
import autogen
import os

config_list = [{"model": "gpt-4o", "api_key": os.environ["OPENAI_API_KEY"]}]

llm_config = {
    "timeout": 600,
    "cache_seed": 42,
    "config_list": config_list,
    "temperature": 0,
}

autogen_agent = autogen.AssistantAgent(
    name="assistant",
    llm_config=llm_config,
)

user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={
        "work_dir": "web",
        "use_docker": False,
    },  # docker를 사용할 수 있는 경우 생성된 코드를 실행하려면 use_docker=True로 설정하세요. docker를 사용하는 것이 생성된 코드를 직접 실행하는 것보다 안전합니다.
    llm_config=llm_config,
    system_message="Reply TERMINATE if the task has been solved at full satisfaction. Otherwise, reply CONTINUE, or the reason why the task is not solved yet.",
)

---

## 워크플로우 생성

이제 AutoGen 에이전트를 호출하는 LangGraph 챗봇 그래프를 만들겠습니다.

In [ ]:
from langchain_core.messages import convert_to_openai_messages, BaseMessage
from langgraph.func import entrypoint, task
from langgraph.graph import add_messages
from langgraph.checkpoint.memory import InMemorySaver


@task
def call_autogen_agent(messages: list[BaseMessage]):
    # openai 스타일 메시지로 변환
    messages = convert_to_openai_messages(messages)
    response = user_proxy.initiate_chat(
        autogen_agent,
        message=messages[-1],
        # 이전 메시지 기록을 컨텍스트로 전달
        carryover=messages[:-1],
    )
    # 에이전트로부터 최종 응답을 가져옵니다
    content = response.chat_history[-1]["content"]
    return {"role": "assistant", "content": content}


# 대화 기록을 저장하기 위한 단기 메모리 추가
checkpointer = InMemorySaver()


@entrypoint(checkpointer=checkpointer)
def workflow(messages: list[BaseMessage], previous: list[BaseMessage]):
    messages = add_messages(previous or [], messages)
    response = call_autogen_agent(messages).result()
    return entrypoint.final(value=response, save=add_messages(messages, response))

## 그래프 실행

이제 그래프를 실행할 수 있습니다.

In [ ]:
# 향후 상호 작용을 위해 에이전트 출력을 유지하려면 스레드 ID를 전달합니다
# highlight-next-line
config = {"configurable": {"thread_id": "1"}}

for chunk in workflow.stream(
    [
        {
            "role": "user",
            "content": "Find numbers between 10 and 30 in fibonacci sequence",
        }
    ],
    # highlight-next-line
    config,
):
    print(chunk)

LangGraph의 [지속성](https://langchain-ai.github.io/langgraph/concepts/persistence/) 기능을 활용하고 있으므로 이제 동일한 스레드 ID를 사용하여 대화를 계속할 수 있습니다 -- LangGraph는 자동으로 이전 기록을 AutoGen 에이전트에 전달합니다:

In [ ]:
for chunk in workflow.stream(
    [
        {
            "role": "user",
            "content": "Multiply the last number by 3",
        }
    ],
    # highlight-next-line
    config,
):
    print(chunk)